In [1]:
import pandas as pd
import numpy as np 
import sys
import re
sys.path.insert(0, "modules")

import numpy as np
import pandas as pd

import plotly.express as px

import matplotlib.pyplot as plt   

import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [2]:
df_data=pd.read_csv('data/dataset.csv')

df_data

,ISIN,BI350,BI493,EG056,EG057,ES018,ES019,ES096,F1754,F1769,...,SR06420,SR06421,SR06422,SR06423,SR08418,SR08419,SR08420,SR08421,SR08422,SR08423
0,CNE100004ZD8,0,NaN,25.644,2.219,NaN,NaN,9.090910,NaN,NaN,...,7.086885929,7.086885929,7.086885929,7.086885929,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156
1,US00108J1097,0,NaN,120.409,47.224,NaN,NaN,NaN,10.0,NaN,...,2.72034812,2.72034812,2.72034812,2.72034812,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961
2,US0079031078,0,NaN,385.733,37.010,NaN,NaN,33.333302,30.0,NaN,...,7.079516888,7.079516888,7.079516888,7.079516888,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782
3,TW0003438008,0,NaN,33.397,7.594,NaN,NaN,42.857101,70.0,NaN,...,7.377765179,7.377765179,7.377765179,7.377765179,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079
4,CNE100003MM9,0,100,64.797,4.375,NaN,0.023,0.000000,60.0,NaN,...,7.086885929,7.086885929,7.086885929,7.086885929,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,JP3990600003,0,NaN,486.909,303.135,NaN,NaN,12.500000,40.0,NaN,...,7.55000782,7.55000782,7.55000782,7.55000782,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321
329,CNE100001R90,0,NaN,182.442,46.045,0.032698,NaN,12.500000,80.0,NaN,...,7.086885929,7.086885929,7.086885929,7.086885929,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407
330,CNE100001DJ8,0,100,139.152,41.178,NaN,NaN,22.222200,70.0,NaN,...,7.086885929,7.086885929,7.086885929,7.086885929,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177
331,US29355A1079,0,100,30.497,3.704,NaN,NaN,14.285700,70.0,NaN,...,2.754028559,2.754028559,2.754028559,2.754028559,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796


In [3]:
df_data.replace("#N/A Field Not Applicable", np.nan, inplace=True)
df_data.replace("#N/A Authorization", np.nan, inplace=True)


C:\Users\Veronica\AppData\Local\Temp\ipykernel_27748\1358017280.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_data.replace("#N/A Field Not Applicable", np.nan, inplace=True)


In [4]:
df_data[df_data.isnull().any(axis=1)]

,ISIN,BI350,BI493,EG056,EG057,ES018,ES019,ES096,F1754,F1769,...,SR06420,SR06421,SR06422,SR06423,SR08418,SR08419,SR08420,SR08421,SR08422,SR08423
0,CNE100004ZD8,0,NaN,25.644,2.219,NaN,NaN,9.090910,NaN,NaN,...,7.086885929,7.086885929,7.086885929,7.086885929,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156
1,US00108J1097,0,NaN,120.409,47.224,NaN,NaN,NaN,10.0,NaN,...,2.72034812,2.72034812,2.72034812,2.72034812,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961
2,US0079031078,0,NaN,385.733,37.010,NaN,NaN,33.333302,30.0,NaN,...,7.079516888,7.079516888,7.079516888,7.079516888,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782
3,TW0003438008,0,NaN,33.397,7.594,NaN,NaN,42.857101,70.0,NaN,...,7.377765179,7.377765179,7.377765179,7.377765179,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079
4,CNE100003MM9,0,100,64.797,4.375,NaN,0.023,0.000000,60.0,NaN,...,7.086885929,7.086885929,7.086885929,7.086885929,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,JP3990600003,0,NaN,486.909,303.135,NaN,NaN,12.500000,40.0,NaN,...,7.55000782,7.55000782,7.55000782,7.55000782,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321
329,CNE100001R90,0,NaN,182.442,46.045,0.032698,NaN,12.500000,80.0,NaN,...,7.086885929,7.086885929,7.086885929,7.086885929,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407
330,CNE100001DJ8,0,100,139.152,41.178,NaN,NaN,22.222200,70.0,NaN,...,7.086885929,7.086885929,7.086885929,7.086885929,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177
331,US29355A1079,0,100,30.497,3.704,NaN,NaN,14.285700,70.0,NaN,...,2.754028559,2.754028559,2.754028559,2.754028559,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796


In [5]:
columns_with_nulls = df_data.columns[df_data.isnull().any()].tolist()
columns_with_nulls

['BI350',
 'BI493',
 'EG056',
 'EG057',
 'ES018',
 'ES019',
 'ES096',
 'F1754',
 'F1769',
 'RX262',
 'IS010',
 'EG030',
 'SR001',
 'SR002',
 'SR003',
 'SR004',
 'SR205',
 'SR217',
 'SR238',
 'SR200',
 'SR211',
 'SR711',
 'SR731',
 'SR727',
 'SR718',
 'SR715',
 'SR005',
 'SR035',
 'SR064',
 'SR084',
 'BI588',
 'BI589',
 'BI590',
 'SR208',
 'SR235',
 'SR231',
 'SR249',
 'ST024',
 'SR752',
 'SR754',
 'SR756',
 'SR760',
 'SR724',
 'SR721',
 'EG03010',
 'EG03011',
 'EG03012',
 'EG03013',
 'EG03014',
 'EG03015',
 'EG03016',
 'EG03017',
 'EG03018',
 'EG03019',
 'EG03020',
 'EG03021',
 'EG03022',
 'EG03023',
 'IS01010',
 'IS01011',
 'IS01012',
 'IS01013',
 'IS01014',
 'IS01015',
 'IS01016',
 'IS01017',
 'IS01018',
 'IS01019',
 'IS01020',
 'IS01021',
 'IS01022',
 'IS01023',
 'ES643',
 'SR00118',
 'SR00119',
 'SR00120',
 'SR00121',
 'SR00122',
 'SR00123',
 'SR00218',
 'SR00219',
 'SR00220',
 'SR00221',
 'SR00222',
 'SR00223',
 'SR00318',
 'SR00319',
 'SR00320',
 'SR00321',
 'SR00322',
 'SR00323'

In [6]:
df_fields=pd.read_csv('data/fields.csv')

df_fields

,ID,FIELD MNEMONIC,FIELD DESCRIPTION,FIELD DEFINITION,UNITS,FREQUENCY,PILLAR,THEME,HISTORY,Created,YEAR,Dashboard
0,BI309,EU_TAX_EST_SC_REV_PCT,EU Taxonomy Estimated SC Revenue Percent,Estimated percentage of total revenue/turnover...,Percentage,Annual,Environmental,N.A.,From 2019,45403,current,N.A.
1,BI310,EU_TAX_EST_SC_REV_AMT,EU Taxonomy Estimated SC Revenue Amount,Estimated amount of revenue/turnover generated...,Million Reporting Currency,Annual,Environmental,N.A.,From 2019,45403,current,N.A.
2,BI311,EU_TAX_EST_ELIGIBLE_REV_PCT,EU Taxonomy Estimated Eligible Revenue Percent,Provides the percentage of total revenue gener...,Percentage,Annual,Environmental,N.A.,From 2019,45312,current,N.A.
3,BI312,EU_TAX_EST_ELIGIBLE_REV_AMT,EU Taxonomy Estimated Eligible Revenue Amount,Provides the amount of revenue/turnover genera...,Million Reporting Currency,Annual,Environmental,N.A.,From 2019,45312,current,N.A.
4,BI313,EU_TAX_EST_SC_ADP_REV_PCT,EU Taxonomy Estimated SC Adaptation Rev Pct,Estimated percentage of total revenue/turnover...,Percentage,Annual,Environmental,Climate Change,From 2019,45617,current,N.A.
...,...,...,...,...,...,...,...,...,...,...,...,...
3656,SR08420,AUDIT_THEME_SCORE,BESG Audit Theme Score,Provides the Bloomberg score evaluating the co...,Score,Ad hoc,Governance,Audit & Oversight,From 2015,44835,2020,G Score
3657,SR08421,AUDIT_THEME_SCORE,BESG Audit Theme Score,Provides the Bloomberg score evaluating the co...,Score,Ad hoc,Governance,Audit & Oversight,From 2015,44835,2021,G Score
3658,SR08422,AUDIT_THEME_SCORE,BESG Audit Theme Score,Provides the Bloomberg score evaluating the co...,Score,Ad hoc,Governance,Audit & Oversight,From 2015,44835,2022,G Score
3659,SR08423,AUDIT_THEME_SCORE,BESG Audit Theme Score,Provides the Bloomberg score evaluating the co...,Score,Ad hoc,Governance,Audit & Oversight,From 2015,44835,2023,G Score


In [7]:
def actualizar_valores(df1, df2):
    # Crear un diccionario para facilitar la búsqueda
    columna_a_buscar = df2.iloc[:, 0]  # Primera columna de la segunda tabla
    ultima_columna = df2.iloc[:, -1]  # Última columna de la segunda tabla
    
    # Recorrer las columnas de la primera tabla
    for columna in df1.columns[1:]:  # Excluir la primera columna (ISIN)
        if columna in columna_a_buscar.values:
            # Obtener el índice de la coincidencia en la segunda tabla
            indice = columna_a_buscar[columna_a_buscar == columna].index[0]
            if ultima_columna[indice] in ['SFDR', 'Scope','Scope/SFDR', 'Net zero', 'posible']:
                # Actualizar los valores en la primera tabla a 'No data'
                df1[columna] = df1[columna].where(df1[columna].notna(), 'No data')
            elif ultima_columna[indice] in ['ESG Score', 'E Score', 'S Score', 'G Score', 'Sales']:
                # Actualizar los valores en la primera tabla a 0
                df1[columna] = df1[columna].where(df1[columna].notna(), 0)
    
    return df1
# Actualizar los valores en la primera tabla
df_data = actualizar_valores(df_data, df_fields)


In [8]:
df_data

,ISIN,BI350,BI493,EG056,EG057,ES018,ES019,ES096,F1754,F1769,...,SR06420,SR06421,SR06422,SR06423,SR08418,SR08419,SR08420,SR08421,SR08422,SR08423
0,CNE100004ZD8,0,No data,25.644,2.219,No data,No data,9.09091,No data,No data,...,7.086885929,7.086885929,7.086885929,7.086885929,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156
1,US00108J1097,0,No data,120.409,47.224,No data,No data,No data,10.0,No data,...,2.72034812,2.72034812,2.72034812,2.72034812,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961
2,US0079031078,0,No data,385.733,37.01,No data,No data,33.333302,30.0,No data,...,7.079516888,7.079516888,7.079516888,7.079516888,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782
3,TW0003438008,0,No data,33.397,7.594,No data,No data,42.857101,70.0,No data,...,7.377765179,7.377765179,7.377765179,7.377765179,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079
4,CNE100003MM9,0,100,64.797,4.375,No data,0.023,0.0,60.0,No data,...,7.086885929,7.086885929,7.086885929,7.086885929,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,JP3990600003,0,No data,486.909,303.135,No data,No data,12.5,40.0,No data,...,7.55000782,7.55000782,7.55000782,7.55000782,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321
329,CNE100001R90,0,No data,182.442,46.045,0.032698,No data,12.5,80.0,No data,...,7.086885929,7.086885929,7.086885929,7.086885929,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407
330,CNE100001DJ8,0,100,139.152,41.178,No data,No data,22.2222,70.0,No data,...,7.086885929,7.086885929,7.086885929,7.086885929,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177
331,US29355A1079,0,100,30.497,3.704,No data,No data,14.2857,70.0,No data,...,2.754028559,2.754028559,2.754028559,2.754028559,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796


In [9]:
df_data[df_data.isnull().any(axis=1)]

,ISIN,BI350,BI493,EG056,EG057,ES018,ES019,ES096,F1754,F1769,...,SR06420,SR06421,SR06422,SR06423,SR08418,SR08419,SR08420,SR08421,SR08422,SR08423


In [10]:
df_company=pd.read_csv('data/companies.csv')

df_company

,ISIN,Name,Member Ticker,Mkt Cap,Industry
0,CNE100004ZD8,ACM RESEARCH S-A,688082 CH Equity,"4,885,443,926.00",Semiconductors
1,US00108J1097,ACM RESEARCH-A,ACMR US Equity,"1,474,910,952.00",Semiconductors
2,US0079031078,ADV MICRO DEVICE,AMD US Equity,"241,546,435,400.00",Semiconductors
3,TW0003438008,ADVANCED ANALOG,3438 TT Equity,"98,788,154.60",Semiconductors
4,CNE100003MM9,ADVANCED MICRO-A,688012 CH Equity,"11,573,338,720.00",Semiconductors
...,...,...,...,...,...
326,JP3990600003,YAC HOLDINGS CO,6298 JP Equity,"137,120,773.60",Semiconductors
327,CNE100001R90,YANGZHOU YANGJ-A,300373 CH Equity,"2,720,872,942.00",Semiconductors
328,CNE100001DJ8,ZHEJIANG JINGS-A,300316 CH Equity,"5,676,078,142.00",Semiconductors
329,US29355A1079,ENPHASE ENERGY INC,ENPH US Equity,"16,138,401,236.00",Semiconductors


In [11]:
merged_df = pd.merge(df_data, df_company[['ISIN', 'Name','Industry','Mkt Cap',]], on='ISIN', how='left')
merged_df 

,ISIN,BI350,BI493,EG056,EG057,ES018,ES019,ES096,F1754,F1769,...,SR06423,SR08418,SR08419,SR08420,SR08421,SR08422,SR08423,Name,Industry,Mkt Cap
0,CNE100004ZD8,0,No data,25.644,2.219,No data,No data,9.09091,No data,No data,...,7.086885929,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156,ACM RESEARCH S-A,Semiconductors,"4,885,443,926.00"
1,US00108J1097,0,No data,120.409,47.224,No data,No data,No data,10.0,No data,...,2.72034812,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961,ACM RESEARCH-A,Semiconductors,"1,474,910,952.00"
2,US0079031078,0,No data,385.733,37.01,No data,No data,33.333302,30.0,No data,...,7.079516888,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782,ADV MICRO DEVICE,Semiconductors,"241,546,435,400.00"
3,TW0003438008,0,No data,33.397,7.594,No data,No data,42.857101,70.0,No data,...,7.377765179,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079,ADVANCED ANALOG,Semiconductors,"98,788,154.60"
4,CNE100003MM9,0,100,64.797,4.375,No data,0.023,0.0,60.0,No data,...,7.086885929,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276,ADVANCED MICRO-A,Semiconductors,"11,573,338,720.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,JP3990600003,0,No data,486.909,303.135,No data,No data,12.5,40.0,No data,...,7.55000782,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321,YAC HOLDINGS CO,Semiconductors,"137,120,773.60"
329,CNE100001R90,0,No data,182.442,46.045,0.032698,No data,12.5,80.0,No data,...,7.086885929,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407,YANGZHOU YANGJ-A,Semiconductors,"2,720,872,942.00"
330,CNE100001DJ8,0,100,139.152,41.178,No data,No data,22.2222,70.0,No data,...,7.086885929,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177,ZHEJIANG JINGS-A,Semiconductors,"5,676,078,142.00"
331,US29355A1079,0,100,30.497,3.704,No data,No data,14.2857,70.0,No data,...,2.754028559,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796,ENPHASE ENERGY INC,Semiconductors,"16,138,401,236.00"


In [12]:
columns_df1 = df_data.columns.tolist()
new_columns_order = columns_df1[:1] +['Name'] +['Mkt Cap']+['Industry'] + columns_df1[1:]
merged_df = merged_df[new_columns_order]
merged_df

,ISIN,Name,Mkt Cap,Industry,BI350,BI493,EG056,EG057,ES018,ES019,...,SR06420,SR06421,SR06422,SR06423,SR08418,SR08419,SR08420,SR08421,SR08422,SR08423
0,CNE100004ZD8,ACM RESEARCH S-A,"4,885,443,926.00",Semiconductors,0,No data,25.644,2.219,No data,No data,...,7.086885929,7.086885929,7.086885929,7.086885929,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156,7.334174156
1,US00108J1097,ACM RESEARCH-A,"1,474,910,952.00",Semiconductors,0,No data,120.409,47.224,No data,No data,...,2.72034812,2.72034812,2.72034812,2.72034812,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961,8.193264961
2,US0079031078,ADV MICRO DEVICE,"241,546,435,400.00",Semiconductors,0,No data,385.733,37.01,No data,No data,...,7.079516888,7.079516888,7.079516888,7.079516888,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782,8.231826782
3,TW0003438008,ADVANCED ANALOG,"98,788,154.60",Semiconductors,0,No data,33.397,7.594,No data,No data,...,7.377765179,7.377765179,7.377765179,7.377765179,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079,7.552586079
4,CNE100003MM9,ADVANCED MICRO-A,"11,573,338,720.00",Semiconductors,0,100,64.797,4.375,No data,0.023,...,7.086885929,7.086885929,7.086885929,7.086885929,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276,6.040477276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,JP3990600003,YAC HOLDINGS CO,"137,120,773.60",Semiconductors,0,No data,486.909,303.135,No data,No data,...,7.55000782,7.55000782,7.55000782,7.55000782,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321,8.540054321
329,CNE100001R90,YANGZHOU YANGJ-A,"2,720,872,942.00",Semiconductors,0,No data,182.442,46.045,0.032698,No data,...,7.086885929,7.086885929,7.086885929,7.086885929,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407,6.53214407
330,CNE100001DJ8,ZHEJIANG JINGS-A,"5,676,078,142.00",Semiconductors,0,100,139.152,41.178,No data,No data,...,7.086885929,7.086885929,7.086885929,7.086885929,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177,5.965545177
331,US29355A1079,ENPHASE ENERGY INC,"16,138,401,236.00",Semiconductors,0,100,30.497,3.704,No data,No data,...,2.754028559,2.754028559,2.754028559,2.754028559,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796,9.261717796


In [14]:
merged_df.to_csv('data/merged_data.csv', index=False)